In [13]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

credentials = DefaultAzureCredential()

ml_client = MLClient(
    credential=credentials,
    subscription_id="a9ea30e6-5728-4375-ac04-cbe4918c3def",
    resource_group_name="rg-training",
    workspace_name="ws-training"
)

In [14]:
my_path = ".data/default_of_credit_card_clients.csv"

v1 = "initial"

my_data = Data(
    name="credit-card",
    version=v1,
    path=my_path,
    description="credit card data",
    type=AssetTypes.URI_FILE
)

try:
    data_asset = ml_client.data.get(name="credit-card", version=v1)
    print(f"Data asset already exists. Name: {my_data.name}, version: {my_data.version}")
except:
    ml_client.data.create_or_update(my_data)
    print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")

Data asset already exists. Name: credit-card, version: initial


In [15]:
import pandas as pd

data_asset = ml_client.data.get(name="credit-card", version=v1)
print(f"Data asset URI: {data_asset.path}")
df = pd.read_csv(data_asset.path)
df.head()

Data asset URI: azureml://subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/resourcegroups/rg-training/workspaces/ws-training/datastores/workspaceblobstore/paths/LocalUpload/f4315d633696cbc4576bf53f59a90e8f/default_of_credit_card_clients.csv


,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


In [16]:
df = pd.read_csv(data_asset.path, header=1)
df.rename(columns={"default payment next month": "default"}, inplace=True)
df.drop("ID", axis=1, inplace=True)
df.to_parquet("./data/cleaned-credit-card.parquet")

In [17]:
import time
v2 = "cleaned" + time.strftime("%Y.%m.%d.%H%M%S", time.gmtime())
my_path = "../get-started-notebooks/data/cleaned-credit-card.parquet"

my_data = Data(
    name="credit-card",
    version=v2,
    path=my_path,
    description="credit card data",
    tags={"training_data": "true", "format": "parquet"},
    type=AssetTypes.URI_FILE
) 

my_data = ml_client.data.create_or_update(my_data)
print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")

Data asset created. Name: credit-card, version: cleaned2024.07.05.001607


In [18]:
import pandas as pd

# get a handle of the data asset and print the URI
data_asset_v1 = ml_client.data.get(name="credit-card", version=v1)
data_asset_v2 = ml_client.data.get(name="credit-card", version=v2)

# print the v1 data
print(f"V1 Data asset URI: {data_asset_v1.path}")
v1df = pd.read_csv(data_asset_v1.path)
print(v1df.head(5))

# print the v2 data
print(
    "_____________________________________________________________________________________________________________\n"
)
print(f"V2 Data asset URI: {data_asset_v2.path}")
v2df = pd.read_parquet(data_asset_v2.path)
print(v2df.head(5))

V1 Data asset URI: azureml://subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/resourcegroups/rg-training/workspaces/ws-training/datastores/workspaceblobstore/paths/LocalUpload/f4315d633696cbc4576bf53f59a90e8f/default_of_credit_card_clients.csv
  Unnamed: 0         X1   X2         X3        X4   X5     X6     X7     X8  \
0         ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3   
1          1      20000    2          2         1   24      2      2     -1   
2          2     120000    2          2         2   26     -1      2      0   
3          3      90000    2          2         2   34      0      0      0   
4          4      50000    2          2         1   37      0      0      0   

      X9  ...        X15        X16        X17       X18       X19       X20  \
0  PAY_4  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3   
1     -1  ...          0          0          0         0       689         0   
2      0  ...       3272       3455    